# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Open csv file
file="Resources/City Data.csv"
df=pd.read_csv(file)
df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,ushuaia,-54.80,-68.30,33.80,86,40,1.45,AR,1592843768
1,port-gentil,-0.72,8.78,78.80,78,90,9.17,GA,1592843718
2,rikitea,-23.12,-134.97,69.60,61,89,15.97,PF,1592843932
3,yellowknife,62.46,-114.35,71.01,63,75,5.82,CA,1592843933
4,vardenis,40.18,45.73,53.13,71,25,4.27,AM,1592843934


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Show the map
gmaps.configure(api_key=g_key)
coord=df[["Lat","Lng"]]
humidity=df["Humidity"]
fig=gmaps.figure()
heatmap_layer=gmaps.heatmap_layer(coord,weights=humidity,dissipating=False, max_intensity=max(df["Humidity"]),point_radius=2)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal_temp=df.loc[(df["Max Temp"]>70)&(df["Max Temp"]<80),:]
ideal_windspeed=ideal_temp.loc[ideal_temp["Wind Speed"]<10,:]
ideal_df=ideal_windspeed.loc[ideal_windspeed["Cloudiness"]==0,:]
ideal_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
93,kasongo-lunda,-6.47,16.82,75.11,25,0,2.15,CD,1592843899
140,issoire,45.55,3.25,77.00,41,0,9.17,FR,1592844103
273,namibe,-15.20,12.15,72.57,69,0,8.25,AO,1592844262
322,grand gaube,-20.01,57.66,72.00,76,0,5.01,MU,1592844317
426,zaltan,32.95,11.87,76.89,55,0,9.55,LY,1592844440
452,premia de mar,41.49,2.37,77.00,84,0,3.00,ES,1592844473
528,andradas,-22.07,-46.57,75.78,45,0,3.24,BR,1592844564


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Google Places API: https://maps.googleapis.com/maps/api/place/nearbysearch/json
from pprint import pprint
hotel_df=ideal_df
base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {"keyword":"hotel","radius":5000,"key":g_key}
hotel_name=[]
for i in range(len(hotel_df["City"])):
    latitude=hotel_df.iloc[i,1]
    longitude=hotel_df.iloc[i,2]
    params["location"]=f"{latitude},{longitude}"
    try:
        hotel=requests.get(base_url,params=params).json()
        hotel_name.append(hotel["results"][0]["name"])
    except:
        hotel_name.append("No hotel found")
hotel_df["Hotel Name"]=hotel_name

C:\Users\Feng\anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [7]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
93,kasongo-lunda,-6.47,16.82,75.11,25,0,2.15,CD,1592843899,No hotel found
140,issoire,45.55,3.25,77.00,41,0,9.17,FR,1592844103,Hôtel le Pariou
273,namibe,-15.20,12.15,72.57,69,0,8.25,AO,1592844262,Hotel ibis Styles Iu Namibe
322,grand gaube,-20.01,57.66,72.00,76,0,5.01,MU,1592844317,Paradise Cove Boutique Hotel
426,zaltan,32.95,11.87,76.89,55,0,9.55,LY,1592844440,No hotel found
452,premia de mar,41.49,2.37,77.00,84,0,3.00,ES,1592844473,Hotel Sorli Emocions
528,andradas,-22.07,-46.57,75.78,45,0,3.24,BR,1592844564,Hotel Dela Inn


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
hotel_layer = gmaps.symbol_layer(locations,info_box_content=hotel_info)

# Display figure
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))